In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)


Map:   0%|          | 0/989 [00:00<?, ? examples/s]

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="my_summarization_model",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    dataloader_pin_memory=True,
    gradient_accumulation_steps=2,  # Gradient accumulation for larger batch size
    lr_scheduler_type="linear"

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.384693,0.195200,0.095800,0.163900,0.163500,19.000000
2,No log,2.317118,0.199700,0.099400,0.168800,0.168500,19.000000
3,No log,2.278642,0.198500,0.099200,0.168800,0.168500,19.000000
4,No log,2.249284,0.200400,0.100500,0.169500,0.169300,19.000000
5,No log,2.232576,0.199500,0.101800,0.170300,0.170000,19.000000
6,No log,2.218928,0.199400,0.100100,0.169200,0.168800,19.000000
7,No log,2.207362,0.199900,0.100900,0.169200,0.168900,19.000000
8,No log,2.201820,0.199500,0.100300,0.168900,0.168500,19.000000
9,2.477600,2.196870,0.200700,0.100300,0.169600,0.169300,19.000000
10,2.477600,2.195593,0.200700,0.100600,0.169900,0.169500,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=620, training_loss=2.4589959913684476, metrics={'train_runtime': 784.173, 'train_samples_per_second': 12.612, 'train_steps_per_second': 0.791, 'total_flos': 2677060833116160.0, 'train_loss': 2.4589959913684476, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1724994814.e07d60ad98dc.507.1:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/manu2501sharma/my_summarization_model/commit/2fbf4553159c819b2e790eaa7ef5269a6fe82af3', commit_message='End of training', commit_description='', oid='2fbf4553159c819b2e790eaa7ef5269a6fe82af3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import pickle
with open('my_summarization_model.pkl', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
text = "Artificial Intelligence and Its Future Impact Artificial intelligence (AI) is rapidly transforming the world, revolutionizing industries and redefining the way humans interact with technology. AI's ability to analyze vast amounts of data and learn from patterns has enabled the development of systems that can perform complex tasks with unprecedented speed and accuracy. From self-driving cars to personalized medicine, AI is at the forefront of innovation. However, the growing capabilities of AI also raise important ethical questions. As AI systems become more autonomous, concerns about job displacement, privacy, and decision-making accountability have come to the forefront. The future of AI will likely involve a delicate balance between maximizing its benefits and addressing these challenges. Governments, businesses, and communities will need to collaborate to ensure that AI is developed and deployed in ways that are ethical, inclusive, and beneficial to all. The next few decades will be crucial in determining how AI shapes the future of humanity, making it imperative for society to engage in thoughtful and informed discussions about its role in our lives."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_billsum_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("stevhliu/my_awesome_billsum_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'by examining vast amounts of data and redefining the way humans interact with technology. With AI becoming more autonomous, concerns about job displacement, privacy, and decision-making accountability have emerged to the forefront. As AI systems become more autonomous, concerns about job displacement, privacy, and decision-making accountability have come to the forefront. However, the growing capabilities of AI raise important ethical questions. The future of AI will likely involve a delicate balance between maximizing its benefits and addressing these'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("pytoch", torch.__version__)
import huggingface_hub
print("huggingface_hub ", huggingface_hub.__version__) #Import the huggingface_hub module
import datasets
import transformers
import evaluate
import accelerate

import numpy
print("datasets ", datasets.__version__)
print("transformers ", transformers.__version__)
print("evaluate ", evaluate.__version__)
print("accelerate ", accelerate.__version__)

print("numpy",  numpy.__version__)

pytoch 2.4.0+cu121
huggingface_hub  0.23.5
datasets  2.21.0
transformers  4.42.4
evaluate  0.4.2
accelerate  0.32.1
numpy 1.26.4
